In [23]:
import pandas as pd
from scipy.stats import ttest_ind
from sklearn.model_selection import GridSearchCV

In [ ]:
import pandas as pd
import glob

file_paths = glob.glob("D:\STAT628-Module3-Group2-main\Climatological-Data\Combined_file.csv")


dataframes = [pd.read_csv(file, parse_dates=['CRSDepTime_timezone', 'CRSArrTime_timezone']) for file in file_paths]


data = pd.concat(dataframes, ignore_index=True)


print(data.shape)
print(data.head())


C:\Users\Kevin\AppData\Local\Temp\ipykernel_36572\2465885457.py:8: DtypeWarning: Columns (6,9,10,11,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  dataframes = [pd.read_csv(file, parse_dates=['CRSDepTime_timezone', 'CRSArrTime_timezone']) for file in file_paths]


(8647592, 36)
  IATA_Code_Operating_Airline        CRSDepTime_timezone  \
0                          EV  2018-01-14 18:35:00-05:00   
1                          EV  2018-01-15 18:45:00-05:00   
2                          EV  2018-01-16 18:35:00-05:00   
3                          EV  2018-01-17 18:35:00-05:00   
4                          EV  2018-01-18 18:45:00-05:00   

         CRSArrTime_timezone Origin Dest  DayOfWeek CancellationCode  \
0  2018-01-14 20:58:00-05:00    EWR  CHS          7              NaN   
1  2018-01-15 21:08:00-05:00    EWR  CHS          1              NaN   
2  2018-01-16 20:58:00-05:00    EWR  CHS          2              NaN   
3  2018-01-17 20:58:00-05:00    EWR  CHS          3              NaN   
4  2018-01-18 21:08:00-05:00    EWR  CHS          4              NaN   

   FlightDate  ArrDelay CRSDepTime_Origin_HourlyVisibility  ...  \
0  2018-01-14       5.0                             16.093  ...   
1  2018-01-15      41.0                             16.093

In [7]:
'''
import pandas as pd
import glob
import os

# Define the columns you want to keep for memory efficiency
columns_to_keep = ['IATA_Code_Operating_Airline', 'CRSDepTime_timezone', 'CRSArrTime_timezone', 'Origin', 'Dest', 
    'DayOfWeek', 'CancellationCode','FlightDate','ArrDelay',
               'CRSDepTime_Origin_HourlyVisibility','CRSDepTime_Dest_HourlyVisibility','CRSArrTime_Dest_HourlyVisibility',
                'CRSDepTime_Origin_HourlyDryBulbTemperature','CRSArrTime_Dest_HourlyDryBulbTemperature',
                'CRSDepTime_Origin_HourlyWindSpeed','CRSArrTime_Dest_HourlyWindSpeed',
               'CRSDepTime_Origin_HourlyRelativeHumidity','CRSArrTime_Dest_HourlyRelativeHumidity',
    
                'CRSDepTime_Origin_HourlySkyConditions_OVC08','CRSArrTime_Dest_HourlySkyConditions_OVC08',
                'CRSDepTime_Origin_HourlySkyConditions_BKN07','CRSArrTime_Dest_HourlySkyConditions_BKN07',
                'CRSDepTime_Origin_HourlySkyConditions_BKN06','CRSArrTime_Dest_HourlySkyConditions_BKN06',
                'CRSDepTime_Origin_HourlySkyConditions_BKN05','CRSArrTime_Dest_HourlySkyConditions_BKN05',
                'CRSDepTime_Origin_HourlySkyConditions_SCT04','CRSArrTime_Dest_HourlySkyConditions_SCT04',
                'CRSDepTime_Origin_HourlySkyConditions_SCT03','CRSArrTime_Dest_HourlySkyConditions_SCT03',
                'CRSDepTime_Origin_HourlySkyConditions_FEW02','CRSArrTime_Dest_HourlySkyConditions_FEW02',
                'CRSDepTime_Origin_HourlySkyConditions_FEW01','CRSArrTime_Dest_HourlySkyConditions_FEW01',
                'CRSDepTime_Origin_HourlySkyConditions_CLR00','CRSArrTime_Dest_HourlySkyConditions_CLR00',
]

# Define the output file name
output_file = 'D:\STAT628-Module3-Group2-main\Climatological-Data\Combined_file.csv'

# Initialize an output file
with open(output_file, 'w') as f_out:
    # Initialize the header_written flag
    header_written = False
    
    # Find all CSV files in the current directory and sort them by filename
    csv_files = sorted(glob.glob("D:\STAT628-Module3-Group2-main\Climatological-Data\Flight-Data-Merged-18/*-*.csv"))  # Sort ensures processing in alphabetical order
    for file in csv_files:
        # Skip the output file if it’s in the same folder
        if file.endswith(output_file):
            print(f"Skipping output file: {file}")
            continue
        
        print(f"Processing file: {file}")  # Print the name of the file being processed
        try:
            # Check if the file is empty or corrupt
            if os.stat(file).st_size == 0:
                print(f"Skipping empty file {file}")
                continue
            
            # Read each file in chunks to manage memory and specify dtype to handle mixed data types
            for chunk in pd.read_csv(file, usecols=columns_to_keep, chunksize=600000, low_memory=False, 
                                     dtype={'IATA_Code_Operating_Airline': str, 'Origin': str, 'Dest': str}):
                
                # Check for missing columns before processing
                missing_columns = set(['CRSDepTime_timezone', 'CRSArrTime_timezone']) - set(chunk.columns)
                if not missing_columns:
                    # Convert time columns to datetime format for calculations (if needed)
                    chunk['CRSDepTime_timezone'] = pd.to_datetime(chunk['CRSDepTime_timezone'], errors='coerce')
                    chunk['CRSArrTime_timezone'] = pd.to_datetime(chunk['CRSArrTime_timezone'], errors='coerce')
                    
                    # Define the correct column order based on columns_to_keep
                    column_order = ['IATA_Code_Operating_Airline', 'CRSDepTime_timezone', 'CRSArrTime_timezone', 'Origin', 'Dest', 
    'DayOfWeek', 'CancellationCode','FlightDate','ArrDelay',
               'CRSDepTime_Origin_HourlyVisibility','CRSDepTime_Dest_HourlyVisibility','CRSArrTime_Dest_HourlyVisibility',
                'CRSDepTime_Origin_HourlyDryBulbTemperature','CRSArrTime_Dest_HourlyDryBulbTemperature',
                'CRSDepTime_Origin_HourlyWindSpeed','CRSArrTime_Dest_HourlyWindSpeed',
               'CRSDepTime_Origin_HourlyRelativeHumidity','CRSArrTime_Dest_HourlyRelativeHumidity',
    
                'CRSDepTime_Origin_HourlySkyConditions_OVC08','CRSArrTime_Dest_HourlySkyConditions_OVC08',
                'CRSDepTime_Origin_HourlySkyConditions_BKN07','CRSArrTime_Dest_HourlySkyConditions_BKN07',
                'CRSDepTime_Origin_HourlySkyConditions_BKN06','CRSArrTime_Dest_HourlySkyConditions_BKN06',
                'CRSDepTime_Origin_HourlySkyConditions_BKN05','CRSArrTime_Dest_HourlySkyConditions_BKN05',
                'CRSDepTime_Origin_HourlySkyConditions_SCT04','CRSArrTime_Dest_HourlySkyConditions_SCT04',
                'CRSDepTime_Origin_HourlySkyConditions_SCT03','CRSArrTime_Dest_HourlySkyConditions_SCT03',
                'CRSDepTime_Origin_HourlySkyConditions_FEW02','CRSArrTime_Dest_HourlySkyConditions_FEW02',
                'CRSDepTime_Origin_HourlySkyConditions_FEW01','CRSArrTime_Dest_HourlySkyConditions_FEW01',
                'CRSDepTime_Origin_HourlySkyConditions_CLR00','CRSArrTime_Dest_HourlySkyConditions_CLR00',
]
                    chunk = chunk[column_order]
                    
                    # Append chunk to the output CSV file
                    chunk.to_csv(f_out, index=False, header=not header_written, mode='a')
                    header_written = True  # Write header only once for the first chunk
                else:
                    print(f"Skipping file {file} as it lacks required columns: {missing_columns}")
        except Exception as e:
            print(f"Error processing file {file}: {e}")
'''

'\nimport pandas as pd\nimport glob\nimport os\n\n# Define the columns you want to keep for memory efficiency\ncolumns_to_keep = [\'IATA_Code_Operating_Airline\', \'CRSDepTime_timezone\', \'CRSArrTime_timezone\', \'Origin\', \'Dest\', \n    \'DayOfWeek\', \'CancellationCode\',\'FlightDate\',\'ArrDelay\',\n               \'CRSDepTime_Origin_HourlyVisibility\',\'CRSDepTime_Dest_HourlyVisibility\',\'CRSArrTime_Dest_HourlyVisibility\',\n                \'CRSDepTime_Origin_HourlyDryBulbTemperature\',\'CRSArrTime_Dest_HourlyDryBulbTemperature\',\n                \'CRSDepTime_Origin_HourlyWindSpeed\',\'CRSArrTime_Dest_HourlyWindSpeed\',\n               \'CRSDepTime_Origin_HourlyRelativeHumidity\',\'CRSArrTime_Dest_HourlyRelativeHumidity\',\n    \n                \'CRSDepTime_Origin_HourlySkyConditions_OVC08\',\'CRSArrTime_Dest_HourlySkyConditions_OVC08\',\n                \'CRSDepTime_Origin_HourlySkyConditions_BKN07\',\'CRSArrTime_Dest_HourlySkyConditions_BKN07\',\n                \'CRSDepTi

In [ ]:
print(data.shape)
print(data.info())



(1048575, 12)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 12 columns):
 #   Column                           Non-Null Count    Dtype  
---  ------                           --------------    -----  
 0   id                               1048575 non-null  int64  
 1   CRSDepTime_timezone              1048575 non-null  object 
 2   CRSArrTime_timezone              1048575 non-null  object 
 3   Marketing_Airline_Network        1048575 non-null  object 
 4   Flight_Number_Marketing_Airline  1048575 non-null  object 
 5   IATA_Code_Operating_Airline      1048575 non-null  object 
 6   Flight_Number_Operating_Airline  1048575 non-null  object 
 7   Origin                           1048575 non-null  object 
 8   Dest                             1048575 non-null  object 
 9   Flight_Duration                  1048575 non-null  object 
 10  Average_ArrDelay                 1048575 non-null  int64  
 11  Cancellation_Rate                104

In [ ]:
#cancelled

data['Cancelled'] = data['CancellationCode'].apply(lambda x: 1 if pd.notnull(x) else 0)
print(data[['CancellationCode', 'Cancelled']].head(5))

  CancellationCode  Cancelled
0              NaN          0
1              NaN          0
2              NaN          0
3              NaN          0
4              NaN          0


In [27]:
# airline type
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()

# 对 'airline_type' 列进行编码
data['airline_type'] = label_encoder.fit_transform(data['IATA_Code_Operating_Airline'])
# 将 'airline_type' 转换为类别变量
data['airline_type'] = data['airline_type'].astype('category')
airline_mapping = pd.DataFrame({
    'Original_Airline_Type': label_encoder.classes_,
    'Encoded_Airline_Type': range(len(label_encoder.classes_))
})

data['DepAirport'] = label_encoder.fit_transform(data['Origin'])
data['DepAirport'] = data['DepAirport'].astype('category')
data['ArrAirport'] = label_encoder.fit_transform(data['Dest'])
data['ArrAirport'] = data['ArrAirport'].astype('category')
# 查看编码结果
print(airline_mapping)


   Original_Airline_Type  Encoded_Airline_Type
0                     9E                     0
1                     AA                     1
2                     AS                     2
3                     B6                     3
4                     C5                     4
5                     DL                     5
6                     F9                     6
7                     G4                     7
8                     G7                     8
9                     HA                     9
10                    MQ                    10
11                    NK                    11
12                    OH                    12
13                    OO                    13
14                    PT                    14
15                    QX                    15
16                    UA                    16
17                    WN                    17
18                    YV                    18
19                    YX                    19
20           

In [ ]:

data['Flightdate'] = pd.to_datetime(data['FlightDate'], errors='coerce')
data['Month'] = data['Flightdate'].dt.month
print(data[['Flightdate', 'Month']].head(5))


KeyError: 'FlightDate'

In [29]:
#flight duration


data['FlightDuration'] = data['CRSArrTime_timezone'] - data['CRSDepTime_timezone']
# 确保 FlightDuration 是 timedelta 类型，然后转换为分钟数
data['FlightDuration_Minutes'] = data['FlightDuration'].apply(lambda x: x.total_seconds() / 60 if pd.notnull(x) else None)

TypeError: unsupported operand type(s) for -: 'str' and 'str'

In [31]:
missing_rate = data['FlightDuration'].isnull().mean()
print(f"Missing rate of 'feature_name': {missing_rate:.2%}")

Missing rate of 'feature_name': 0.00%


In [ ]:
#Time of day

def get_time_of_day(time):
    hour = time.hour
    if 0 <= hour < 6:
        return 0   #  late night
    elif 6 <= hour < 11:
        return 1   #  morning
    elif 11 <= hour < 13:
        return 2   #  noon
    elif 13 <= hour < 18:
        return 3   # afternoon
    elif 18 <= hour < 24:
        return 4   # night


data['CRSTimeOfDay_Dep'] = data['CRSDepTime_timezone'].apply(get_time_of_day)
data['CRSTimeOfDay_Arr'] = data['CRSArrTime_timezone'].apply(get_time_of_day)
data['CRSTimeOfDay_Dep'] = data['CRSTimeOfDay_Dep'].astype('category')
data['CRSTimeOfDay_Arr'] = data['CRSTimeOfDay_Arr'].astype('category')

print(data[['FlightDuration_Minutes', 'CRSTimeOfDay_Dep','CRSTimeOfDay_Arr']].head())





C:\Users\Kevin\AppData\Local\Temp\ipykernel_36572\3256607117.py:6: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise a warning unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  data['CRSDepTime_timezone'] = pd.to_datetime(data['CRSDepTime_timezone'], errors='coerce')
C:\Users\Kevin\AppData\Local\Temp\ipykernel_36572\3256607117.py:7: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise a warning unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  data['CRSArrTime_timezone'] = pd.to_datetime(data['CRSArrTime_timezone'], errors='coerce')


   FlightDuration_Minutes CRSTimeOfDay_Dep CRSTimeOfDay_Arr
0                   204.0                4                4
1                    84.0                3                3
2                   116.0                3                4
3                   245.0                1                3
4                   336.0                1                3


In [ ]:
#Skycond

def determine_CRSDepTimeskycond(row):
    if row['CRSDepTime_Origin_HourlySkyConditions_OVC08'] == 1:
        return 4
    elif (row['CRSDepTime_Origin_HourlySkyConditions_BKN05'] == 1 or
          row['CRSDepTime_Origin_HourlySkyConditions_BKN06'] == 1 or
          row['CRSDepTime_Origin_HourlySkyConditions_BKN07'] == 1 ):
        return 3
    elif (row['CRSDepTime_Origin_HourlySkyConditions_SCT03'] == 1 or
          row['CRSDepTime_Origin_HourlySkyConditions_SCT04'] == 1 ):
        return 2
    elif (row['CRSDepTime_Origin_HourlySkyConditions_FEW01'] == 1 or
          row['CRSDepTime_Origin_HourlySkyConditions_FEW02'] == 1):
        return 1
    elif row['CRSDepTime_Origin_HourlySkyConditions_CLR00'] == 1:
        return 0
    else:
        return None  
def determine_CRSArrTimeskycond(row):
    if row['CRSArrTime_Dest_HourlySkyConditions_OVC08'] == 1:
        return 4
    elif (row['CRSArrTime_Dest_HourlySkyConditions_BKN05'] == 1 or
          row['CRSArrTime_Dest_HourlySkyConditions_BKN06'] == 1 or
          row['CRSArrTime_Dest_HourlySkyConditions_BKN07'] == 1 ):
        return 3
    elif (row['CRSArrTime_Dest_HourlySkyConditions_SCT03'] == 1 or
          row['CRSArrTime_Dest_HourlySkyConditions_SCT04'] == 1 ):
        return 2
    elif (row['CRSArrTime_Dest_HourlySkyConditions_FEW01'] == 1 or
          row['CRSArrTime_Dest_HourlySkyConditions_FEW02'] == 1):
        return 1
    elif row['CRSArrTime_Dest_HourlySkyConditions_CLR00'] == 1:
        return 0
    else:
        return None  

data['CRSDepTime_Origin_skycond'] = data.apply(determine_CRSDepTimeskycond, axis=1)
data['CRSArrTime_Dest_skycond'] = data.apply(determine_CRSArrTimeskycond, axis=1)



In [19]:
data['DayOfWeek'] = data['DayOfWeek'].astype('category')

In [ ]:
crs_columns = ['airline_type','FlightDuration_Minutes','DepAirport','ArrAirport','DayOfWeek',
               'CRSTimeOfDay_Dep','CRSTimeOfDay_Arr', 'Month',
                'CRSDepTime_Origin_skycond',   'CRSArrTime_Dest_skycond'        ,                             # skycondition
               'CRSDepTime_Origin_HourlyRelativeHumidity', 'CRSArrTime_Dest_HourlyRelativeHumidity' ,   #humidity
               'CRSDepTime_Origin_HourlyDryBulbTemperature','CRSArrTime_Dest_HourlyDryBulbTemperature' ,      #temperature    
               'CRSDepTime_Origin_HourlyWindSpeed',  'CRSArrTime_Dest_HourlyWindSpeed',  # windspeed
                'CRSDepTime_Origin_HourlyVisibility', 'CRSArrTime_Dest_HourlyVisibility', # visibility  
                'Cancelled','ArrDelay'
                ]
final_data = data[crs_columns]

final_data = final_data.dropna(subset=[col for col in final_data.columns if col != 'ArrDelay'])

final_data.to_csv('final_data.csv', index=False)